# Import libraries

In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("ticks")

from cleaning_pipeline import *

# Load data

In [ ]:
df = pd.read_csv("data/WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [ ]:
df.head()

# Clean data

In [ ]:
cleaned_df = (
    df.pipe(start_pipeline)
    .pipe(drop_noisy_columns, cols=["customerID"])
    .pipe(replace_empty_strings_with_nan)
)

In [ ]:
cleaned_df.head()

In [ ]:
cleaned_df.isna().sum() / len(df)

We can see that the percentage of missing values in the `TotalCharges` column is less than 1%, so we can drop all rows with missing values.

In [ ]:
cleaned_df = cleaned_df.pipe(drop_missing_values)

In [ ]:
cleaned_df

Now we can convert each column to the appropriate `dtype`.

In [ ]:
cleaned_df = cleaned_df.pipe(
    convert_column_dtypes, {"SeniorCitizen": "str", "TotalCharges": np.float}
)

In [ ]:
cleaned_df.head()

In [ ]:
cleaned_df.dtypes

Our final cleaning and processing pipeline should be:

In [ ]:
cleaned_df = (
    df.pipe(start_pipeline)
    .pipe(drop_noisy_columns, cols=["customerID"])
    .pipe(replace_empty_strings_with_nan)
    .pipe(drop_missing_values)
    .pipe(map_column_values, col="SeniorCitizen", mapping_dict={0: "No", 1: "Yes"})
    .pipe(convert_column_dtypes, dtypes_mapping={"TotalCharges": np.float},)
)

In [ ]:
cleaned_df.head()

# Exploratory Data Analysis

In [ ]:
demographic_cols = [
    "gender",
    "SeniorCitizen",
    "Partner",
    "Dependents",
]

In [ ]:
account_cols = [
    "tenure",
    "Contract",
    "PaymentMethod",
    "PaperlessBilling",
    "MonthlyCharges",
    "TotalCharges",
]

In [ ]:
services_cols = [
    "PhoneService",
    "MultipleLines",
    "InternetService",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
]

In [ ]:
internet_service_cols = [
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
]

In [ ]:
color_map = {"Yes": "#ef553b", "No": "#636efa"}

## Target variable

In [ ]:
fig = px.histogram(
    data_frame=cleaned_df, x="Churn", color="Churn", color_discrete_map=color_map,
)

fig.show()

## Demographic attributes

### `Gender`

In [ ]:
fig = px.histogram(
    data_frame=cleaned_df, x="gender", color="Churn", color_discrete_map=color_map
)

fig.update_xaxes(categoryorder="total descending")

fig.show()

### `SeniorCitizen`

In [ ]:
fig = px.histogram(
    data_frame=cleaned_df,
    x="SeniorCitizen",
    color="Churn",
    color_discrete_map=color_map,
)

fig.show()

### `Partner`

In [ ]:
fig = px.histogram(
    data_frame=cleaned_df, x="Partner", color="Churn", color_discrete_map=color_map
)

fig.update_xaxes(categoryorder="total descending")

fig.show()

### `Dependents`

In [ ]:
fig = px.histogram(
    data_frame=cleaned_df, x="Dependents", color="Churn", color_discrete_map=color_map
)

fig.show()

## Account attributes

### `tenure`

In [ ]:
cleaned_df.tenure.head()

In [ ]:
df.tenure.describe()

In [ ]:
df.tenure.skew()

In [ ]:
fig = px.histogram(data_frame=cleaned_df, x="tenure", marginal="box")

fig.show()

In [ ]:
fig = px.histogram(
    data_frame=cleaned_df,
    x="tenure",
    color="Churn",
    barmode="group",
    marginal="box",
    color_discrete_map=color_map,
)

fig.show()

We can conclude from this plot some important insights:
- New customers with about 3 months of tenure are the most likely to churn.
- The chart also shows that the higher customer tenure is, the less likely is he to churn.
- There are some extreme points (outliers) for the *churning* customers, where they have a very high tenure (70 months, almost 6 years), yet they churn from the company. These might be *outliers*, or this could be related to other factors.

### `MonthlyCharges`

In [ ]:
cleaned_df.MonthlyCharges.describe()

In [ ]:
cleaned_df.MonthlyCharges.skew()

In [ ]:
fig = px.histogram(
    data_frame=cleaned_df,
    x="MonthlyCharges",
    marginal="box",
    color="Churn",
    barmode="group",
    color_discrete_map=color_map,
)

fig.show()

### `TotalCharges`

In [ ]:
cleaned_df.TotalCharges.describe()

In [ ]:
cleaned_df.TotalCharges.skew()

In [ ]:
fig = px.histogram(
    data_frame=cleaned_df,
    x="TotalCharges",
    marginal="box",
    color="Churn",
    barmode="group",
    color_discrete_map=color_map,
)

fig.show()

### `Contract`

In [ ]:
fig = px.histogram(data_frame=cleaned_df, x="Contract", color="Contract")

fig.update_xaxes(categoryorder="total descending")

fig.show()

The majority of customers prefer *monthly* contracts.

Let's see the relation between contract type and whether customer churn the company or not:

In [ ]:
fig = px.histogram(
    data_frame=cleaned_df,
    x="Contract",
    color="Churn",
    barmode="group",
    color_discrete_map=color_map,
)

fig.update_xaxes(categoryorder="total descending")

fig.show()

This chart suggests that customers who use short term contracts are far more likely to leave the service.

### `PaymentMethod`

In [ ]:
fig = px.histogram(data_frame=cleaned_df, x="PaymentMethod", color="PaymentMethod")

fig.update_xaxes(categoryorder="total descending")

fig.show()

In [ ]:
fig = px.histogram(data_frame=cleaned_df, x="PaymentMethod", color="Churn")

fig.update_xaxes(categoryorder="total descending")

fig.show()

Customers who use *Electronic check* payment method are more likely to stop using the service.

### `PaperlessBilling`

In [ ]:
fig = px.histogram(
    data_frame=cleaned_df,
    x="PaperlessBilling",
    color="Churn",
    color_discrete_map=color_map,
)

fig.show()

## Services attributes

### `PhoneService`

In [ ]:
cleaned_df.PhoneService.value_counts()

In [ ]:
fig = px.histogram(
    data_frame=cleaned_df, x="PhoneService", color="Churn", color_discrete_map=color_map
)

fig.update_xaxes(categoryorder="total descending")

fig.show()

### `MultipleLines`

In [ ]:
cleaned_df.MultipleLines.value_counts(normalize=True)

In [ ]:
fig = px.histogram(
    data_frame=cleaned_df,
    x="MultipleLines",
    color="Churn",
    color_discrete_map=color_map,
)

fig.update_xaxes(categoryorder="total descending")

fig.show()

### `InternetService`

In [ ]:
cleaned_df.InternetService.value_counts()

In [ ]:
fig = px.histogram(data_frame=cleaned_df, x="InternetService", color="InternetService")

fig.update_xaxes(categoryorder="total descending")

fig.show()

In [ ]:
fig = px.histogram(data_frame=cleaned_df, x="InternetService", color="Churn")

fig.update_xaxes(categoryorder="total descending")

fig.show()

In [ ]:
fig = px.histogram(
    data_frame=cleaned_df,
    x="MonthlyCharges",
    color="Churn",
    barmode="group",
    facet_row="InternetService",
    height=1000,
)

fig.show()

In [ ]:
fig = px.histogram(
    data_frame=cleaned_df,
    x="MonthlyCharges",
    color="InternetService",
    barmode="group",
    facet_row="Churn",
)

fig.show()

This chart shows that, on general, customers whoe are using the `Fiber optic` internet service are paying more monthly charges than customers who are using `DSL`, and of course higher than customer who don't use internet service at all.

The chart also shows that among the customers who stop using the company services, the fiber optic users are the most common. This might be linked to somer characteristics of the fiber optic service: price, quality comapred with other companies, etc ...

### other services

Other internet services include:
- `OnlineSecurity`
- `OnlineBackup`
- `DeviceProtection`
- `TechSupport`
- `StreamingTV`
- `StreamingMovies`

In [ ]:
internet_services_df = pd.melt(
    cleaned_df.loc[cleaned_df.InternetService != "No", internet_service_cols]
).rename(columns={"variable": "service", "value": "HasService"})

In [ ]:
internet_services_df.head()

In [ ]:
fig = px.histogram(data_frame=internet_services_df, x="service", color="HasService")

fig.show()

# Focusing on recent customers

We saw earlier from the `tenure` variable distribution that new customers (the ones that have been using the company's services for less than a year) are more likely to churn.

In this section, we'll take a closer look at these customers, to understand their behavior and characteristics.

The `tenure` variable is measured in number of months. so let's keep only the customers who stayed in the company for a 12 months (a year):

In [ ]:
one_year_customers_df = cleaned_df[cleaned_df.tenure <= 12]

In [ ]:
len(one_year_customers_df) / len(cleaned_df)

30% of customers stayed in the company for a year.

## How many of recent customers did churn?

In [ ]:
fig = px.histogram(
    data_frame=one_year_customers_df,
    x="Churn",
    color="Churn",
    color_discrete_map=color_map,
    title="Churn distribution for new customers",
)

fig.show()

Almost half of the recent customers have churned.

## The monthly charges of recent customers:

In [ ]:
fig = px.histogram(
    data_frame=one_year_customers_df,
    x="MonthlyCharges",
    color="Churn",
    color_discrete_map=color_map,
    barmode="group",
    marginal="box",
    nbins=10,
    title="Distribution of monthly charges for new customers",
)

fig.show()

The higher new customers pay per month, the more likely they churn from the company.

## What contract type did recent customers prefer?

In [ ]:
fig = px.histogram(
    data_frame=one_year_customers_df,
    x="Contract",
    color="Churn",
    title="Contract type for new customers",
)

fig.show()

It seems that the `month-to-month` contract is the most prefered type of contract for new customers.

We can also see customers who use this type of contract are more likely to churn.

## What type of services did recent customer use?

In this section we'll investigate what type of services did recent customers choose to use, and the relation between different services and how much the customer pay per month.

All that will help us identify any services that are more expensive compared to other services, or caused the customer any dissatisfaction, which leads them to churn from the company.

### `PhoneService`

In [ ]:
one_year_customers_df.PhoneService.value_counts()

In [ ]:
fig = px.histogram(
    data_frame=one_year_customers_df,
    x="PhoneService",
    color="Churn",
    color_discrete_map=color_map,
)

fig.update_xaxes(categoryorder="total descending")

fig.show()

### `MultipleLines`

In [ ]:
one_year_customers_df.MultipleLines.value_counts()

In [ ]:
fig = px.histogram(
    data_frame=one_year_customers_df,
    x="MultipleLines",
    color="Churn",
    color_discrete_map=color_map,
)

fig.update_xaxes(categoryorder="total descending")

fig.show()

### `InternetService`

In [ ]:
one_year_customers_df.InternetService.value_counts()

In [ ]:
fig = px.histogram(
    data_frame=one_year_customers_df,
    x="InternetService",
    color="Churn",
    color_discrete_map=color_map,
)

fig.update_xaxes(categoryorder="total descending")

fig.show()

### other services

In [ ]:
one_year_internet_services_df = pd.melt(
    one_year_customers_df.loc[
        one_year_customers_df.InternetService != "No", internet_service_cols
    ]
).rename(columns={"variable": "service", "value": "HasService"})

In [ ]:
one_year_internet_services_df.head()

In [ ]:
fig = px.histogram(data_frame=one_year_internet_services_df, x='service', color='HasService')

fig.show()